In [1]:
import iris
import numpy
from pyproj import Proj
from shapely.geometry import shape

## Read the data

The idea is to use iris so that the `cube.coord('latitude').guess_bounds()` method can be used if the bounds aren't supplied. (This can be done for latitude, longitude or level). The values held in an iris cube are `numpy.ma.core.MaskedArray`.

In [2]:
infile = "/Users/irv033/Downloads/data/thetao_Omon_CSIRO-Mk3-6-0_piControl_r1i1p1_000101-001012.nc"

In [3]:
def lat_subset(cell):
    return -10 < cell < 10

def lon_subset(cell):
    return 30 < cell < 70

lat_constraint = iris.Constraint(latitude=lat_subset)
lon_constraint = iris.Constraint(longitude=lon_subset)

In [4]:
with iris.FUTURE.context(cell_datetime_objects=True):
    cube = iris.load_cube(infile, 'sea_water_potential_temperature' & lat_constraint & lon_constraint)

/Users/irv033/miniconda2/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:794: UserWarning: Missing CF-netCDF measure variable u'areacello', referenced by netCDF variable u'thetao'
  warnings.warn(message % (variable_name, nc_var_name))
/Users/irv033/miniconda2/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:794: UserWarning: Missing CF-netCDF measure variable u'volcello', referenced by netCDF variable u'thetao'
  warnings.warn(message % (variable_name, nc_var_name))
/Users/irv033/miniconda2/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:1139: UserWarning: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warnings.warn(msg)


In [5]:
print cube

sea_water_potential_temperature / (K) (time: 120; depth: 31; latitude: 20; longitude: 20)
     Dimension coordinates:
          time                             x           -             -              -
          depth                            -           x             -              -
          latitude                         -           -             x              -
          longitude                        -           -             -              x
     Attributes:
          Conventions: CF-1.4
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CSIRO-Mk3-6-0_piControl_r0i0p0.nc...
          branch_time: 0.0
          cmor_version: 2.5.9
          comment: Data is stored on the native ocean T-grid on which the data was generated....
          contact: Project leaders: Stephen Jeffrey (Stephen.Jeffrey@qld.gov.au) & Leon Rotstayn...
          creation_date: 2011-05-11T07:04:01Z
          experiment: pre-industrial co

In [6]:
lat_bounds = cube.coord('latitude').bounds
lon_bounds = cube.coord('longitude').bounds
lev_bounds = cube.coord('depth').bounds

print lat_bounds.shape
print lon_bounds.shape
print lev_bounds.shape

print type(lat_bounds)

(20, 2)
(20, 2)
(31, 2)
<type 'numpy.ndarray'>


In [7]:
lat_diffs = numpy.apply_along_axis(lambda x: x[1] - x[0], 1, lat_bounds)
lon_diffs = numpy.apply_along_axis(lambda x: x[1] - x[0], 1, lon_bounds)
lev_diffs = numpy.apply_along_axis(lambda x: x[1] - x[0], 1, lev_bounds)

In [8]:
print lat_diffs.shape
print lat_diffs[3]
print lat_bounds[3,:]

(20,)
0.932671070099
[-6.5287075  -5.59603643]


## Perform the integration

Rather than use [`scipy.integrate.simps`](http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.integrate.simps.html) or something similar, I think it's best to use the level bounds provided in the dataset to essentially define and calculate the area of a series of rectangles.

#### Single point

In [9]:
test_point = cube.data[1, :, 1, 1]
print test_point
print type(test_point)

[-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- --]
<class 'numpy.ma.core.MaskedArray'>


In [10]:
areas = test_point * lev_diffs
print areas
print areas.sum()

[-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- --]
--


In [11]:
test_point = cube.data[1, :, 15, 1]
print test_point
print type(test_point)

[-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- --]
<class 'numpy.ma.core.MaskedArray'>


In [12]:
areas = test_point * lev_diffs
print areas
print areas.sum()

[-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- --]
--


#### Whole array

In [13]:
def simple_integration(vector, lev_steps):
    """Perform integration for a single vector (of length of level dimension)"""
    
    areas = vector * lev_steps
    
    return areas.sum()

In [14]:
OHC_per_m2 = numpy.ma.apply_along_axis(simple_integration, 1, cube.data, lev_diffs)

In [15]:
print OHC_per_m2.shape
print type(OHC_per_m2)
print OHC_per_m2[1, 1, 1]
print OHC_per_m2[1, 15, 1]

(120, 20, 20)
<class 'numpy.ma.core.MaskedArray'>
--
--


It is at this point that I might save the field to file (i.e. with units $J / m^2$). Spatial plots of OHC usually have these units.

## Integrating up: Multiply by the area

### Example

Following [this](http://stackoverflow.com/questions/4681737/how-to-calculate-the-area-of-a-polygon-on-the-earths-surface-using-python) Stack Overflow response...

In [18]:
co = {"type": "Polygon", "coordinates": [
    [(-102.05, 41.0),
     (-102.05, 37.0),
     (-109.05, 37.0),
     (-109.05, 41.0)]]}
lon, lat = zip(*co['coordinates'][0])
pa = Proj("+proj=aea +lat_1=37.0 +lat_2=41.0 +lat_0=39.0 +lon_0=-106.55")

In [19]:
x, y = pa(lon, lat)
cop = {"type": "Polygon", "coordinates": [zip(x, y)]}
shape(cop).area 

268952044107.43454

In [20]:
co['coordinates'][0]

[(-102.05, 41.0), (-102.05, 37.0), (-109.05, 37.0), (-109.05, 41.0)]

In [21]:
print lat
print lon

(41.0, 37.0, 37.0, 41.0)
(-102.05, -102.05, -109.05, -109.05)


### My data

In [22]:
eg_lat = lat_bounds[0, :]
eg_lon = lon_bounds[0, :]

print eg_lat
print eg_lon

[-9.3267231  -8.39405155]
[ 30.9375  32.8125]


In [23]:
mesh_lat, mesh_lon = numpy.meshgrid(eg_lat, eg_lon)

print mesh_lat.flatten()
print mesh_lon.flatten()

[-9.3267231  -8.39405155 -9.3267231  -8.39405155]
[ 30.9375  30.9375  32.8125  32.8125]


In [38]:
def cell_area(lon_bnds, lat_bnds):
    """Grid cell area."""
    
    lons = [lon_bnds[0], lon_bnds[0], lon_bnds[1], lon_bnds[1]]
    lats = [lat_bnds[0], lat_bnds[1], lat_bnds[1], lat_bnds[0]]
    
    ave_lon = (lon_bnds[0] + lon_bnds[-1]) / 2.0
    ave_lat = (lat_bnds[0] + lat_bnds[-1]) / 2.0
    proj_syntax = "+proj=aea +lat_1=%f +lat_2=%f +lat_0=%f +lon_0=%f" %(lat_bnds[0], lat_bnds[-1], ave_lat, ave_lon)
    
    print proj_syntax
    
    pa = Proj(proj_syntax)
    
    x, y = pa(lons, lats)
    cop = {"type": "Polygon", "coordinates": [zip(x, y)]}
    
    return shape(cop).area

In [39]:
cell_area(eg_lon, eg_lat)

+proj=aea +lat_1=-9.326723 +lat_2=-8.394052 +lat_0=-8.860387 +lon_0=31.875000


21275166772.52099

In [40]:
lon_diff = eg_lon[-1] - eg_lon[0]
lat_diff = eg_lat[-1] - eg_lat[0]

In [41]:
print lon_diff
print lat_diff

1.875
0.932671546936


Check the calculation with figures from [here](http://fas.org/news/reference/calc/degree.html)...

In [42]:
deg_lat_m = 110601
deg_lon_m = 109958

(deg_lat_m * lat_diff) * (deg_lon_m * lon_diff)

21267472779.097378